In [ ]:
# TEST

In [ ]:
"press 'alt + \' to trigger inline (ai - copilot) suggestions"

import sqlite3
class database_manager:
    def __init__(self, db_name = "database.db"):
        self.db_name = db_name
        self.init_database()

    def init_database(self):
        with sqlite3.connect(self.db_name) as connect:
            cursor = connect.cursor()
            cursor.execute ('''
                           CREATE TABLE IF NOT EXISTS entity (
                           id INTEGER PRIMARY KEY AUTOINCREMNT,
                           name TEXT NOT NULL
                           email UNIQUE NOT NULL,
                           age INTEGER NOT NULL,
                           IC UNIQUE NOT NULL,
                           created_by TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                           )
                           ''')
            cursor.execute ('''
                           CREATE TABLE IF NOT EXISTS records (
                           id INTEGER PRIMARY KEY AUTOINCREMENT,
                           user_id INTEGER NOT NULL,
                           IC UNIQUE NOT NULL,
                           email UNIQUE NOT NULL,
                           remarks TEXT,
                           created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                           FOREIGN KEY (user_id) REFERENCES entity (id)
                           )
                           ''')
            
    def create_user(self, name, email, age, IC): # Creates new user
        try:
            with sqlite3.connect(self.db_name) as connect:
                cursor = connect.cursor()
                cursor.execute('''
                               INSERT INTO entity (name, email, age, ID)
                               VALUES (?, ?, ?, ?)
                               ''', (nam , email, age, IC))
                return cursor.lastrowid
        except sqlite3.IntegrityError as err:
            print(f"Error = {err}")
            return None
        
    def create_records (self, user_id, title, content): # Creates new record
        try:
            with sqlite3.connect(self.db_name) as connect:
                cursor = connect.cursor()
                cursor.execute('''
                            INSERT INTO records (user_id, title, remarks)
                            VALUES (?, ?, ?)
                            ''', (user_id, title, remarks))
                return cursor.lastrowid
        except sqlite3.IntegrityError as err:
            print(f"Error = {err}")
            return None
        
    def call_every_entity(self): # Gets all users
        with sqlite3.connect(self.db_name) as connect:
            cursor  = connect.cursor()
            cursor.execute('SELECT * FROM entity')
            return cursor.fetchall()
        
    def call_entity_record(self, user_id): # Gets record
        with sqlite3.connect(self.db_name) as connect:
            cursor = connect.cursor()
            cursor.execute('''SELECT r.id, r.title, r.remarks, r.created_at
                        FROM records r
                        WHERE r.user_id = ?
                        ORDER BY r.creataed_at DESC
                        ''', (user_id,))
            return cursor.fetchall()
    
    def remove_entity(self, user_id): # Deletes a user
        with sqlite3.connect(self.db_name) as connect:
            cursor = connect.cursur()
            cursor.execute('DELETE FROM records WHERE user_id = ?', (user_id))
            cursur.execute('DELETE FROM entity WHERE id = ?', (user_id,))
            return cursor.rowcount > 0
        
def display_terminal_menu(): # Displays the menu in terminal
    print("\n" + "="*40)
    print("    DATABASE MANAGER")
    print("="*40)
    print("1 CREATE USER")
    print("2 VIEW ALL USER")
    print("3 CREATE RECORD")
    print("4 VIEW USER RECORD")
    print("5 DELETE USER")
    print("6 EXIT")
    print("="*40)

def main(): # Main function
    db = database_manager()
    while True:
        display_terminal_menu()
        choice = input("ENTER YOUR CHOICE: ").strip()

        if choice == "1":
            print("\n--- Create New User ---")
            name = input("Enter name: ")
            email = input("Enter email: ")
            try:
                age = int(input("Enter age: ").strip())
                user_id = db.create_user(name, email, age)
                if user_id:
                    print(f"Success ID: {user_id}")
                else:
                    print("INVALID ID")
            except ValueError:
                print("invalid AGE format")

        elif choice == 2:
            print("\n--- Create New User ---")
            user = db.get_all_user()
            if user:
                for u in user:
                    print(f"")
                    db.get_all_users(f"Name: {user[1]} | Email: {user[2]} | Age: {user[3]}")
            else:
                print("N/A")

        elif choice == 3:
            print("\n--- Create New Record ---")
            try:
                user_id = int(input("Enter user ID: ").strip())
                title =  input("Enter user title: ")
                remarks = input("Enter record remarks: ")
                post_id = db.get_all_user()
                if user:
                    for u in user:
                        print(f"")
                        db.get_all_users(f"Name: {user[1]} | Email: {user[2]} | Age: {user[3]}")
                else:
                    print("N/A")
            except ValueError:
                print("Invalid user ID")

        elif choice == 4:
            print("\n--- View User Records ---")
            try:
                user_id = int(input("Enter user ID: ").strip())
                records =  db.get_user_records(user_id)
                if records:
                    for re in records:
                        print(f"\nRecord ID: {records[0]}")
                        print(f"Title: {records[1]}")
                        print(f"Content: {records[2]}")
                        print(f"created_at: {records[3]}")
                        print("-"*30)
                else:
                    print("No record found for said user")
            except ValueError:
                print("Invalid user ID")

        elif choice == 5:
            print("\n--- Delete User ---")
            try:
                user_id = int(input("Enter user ID to delete: ").strip())
                confirm = input(f"Confrim delete user {user_id}").strip().lower()
                if confirm == "y":
                    if db.delete_user(user_id):
                        print("User deleted")
                    else:
                        print("User N/A")
                else:
                    print("Opeation cancelled")
            except ValueError:
                print("Invalid user ID")

        elif choice == 6:
            print("\n--- Delete User ---")
            break

        else:
            print("Invalid choce")
        input("\nPress any key to continue ...")

if 

In [ ]:
from pymongo import MongoClient
from datetime import datetime
from bson.objectid import ObjectId
from dotenv import load_dotenv
import os

load_dotenv() # Load environment variables from .env file

mongo_uri = os.getenv('MONGODB_ATLAS_CLUSTER_URI')

class DatabaseManager:
    def __init__(self, db_name='example_db', connection_string=mongo_uri):
        self.client = MongoClient(connection_string)
        self.db = self.client[db_name]
        self.users_collection = self.db.users
        self.posts_collection = self.db.posts
        self.init_database()

    def init_database(self): # Initialize database with collections and indexes
        self.users_collection.create_index("email", unique=True)    # Create unique index on email for users
        self.posts_collection.create_index("user id")               # Create index on user_id for posts for better query performance

    def create_user(self, name, email, age): # Create a new user
        try:
            user_doc = {
            "name": name,
            "email": email,
            "age": age,
            "created_at": datetime.now()
            }
            result = self.users_collection.insert_one(user_doc)
            return str(result.inserted_id)
        except Exception as e:
            print(f"Error: {e}")
            return None

    def create_post(self, user_id, title, content): # Create a new post
        try:
            if ObjectId.is_valid(user_id):          # Convert string user_id to ObjectId if it's a valid ObjectId
                user_object_id = ObjectId(user_id)
            else:
                user_object_id = user_id

            post_doc = {
                "user_id": user_object_id,
                "title": title,
                "content": content,
                "created_at": datetime.now()
                }
            result = self.posts_collection.insert_one(post_doc)
            return str(result.inserted_id)
        except Exception as e:
            print(f"Error creating post: {e}")
            return None
        
    def get_all_users(self):                        #Get all users
        try:
            users = list(self.users_collection.find())
            for user in users:                      # Convert ObjectId to string for display
                user['_id'] = str(user['_id'])
            return users
        except Exception as e:
            print(f"Error fetching users: {e}")
            return []

    def get_user_posts(self, user_id):              #Get posts by user
        try:                                        # Convert string user_id to ObjectId if it's a valid ObjectId
            if ObjectId.is_valid(user_id):
                user_object_id = ObjectId(user_id)
            else:
                user_object_id = user_id

            posts = list(self.posts_collection.find(
            {"user_id": user_object_id}
            ).sort("created_at", -1))

            for post in posts:                      # Convert ObjectId to string for display
                post['_id'] = str(post['_id'])
                post['user_id'] = str(post['user_id'])

            return posts
        
        except Exception as e:
            print(f"Error fetching posts: {e}")
            return []
        
    def delete_user(self, user_id):                 # Delete user and their posts
        try:                                        # Convert string user_id to ObjectId if it's a valid ObjectId
            if ObjectId.is_valid(user_id):
                user_object_id = ObjectId(user_id)
            else:
                user_object_id = user_id

            self.posts_collection.delete_many({"user_id": user_object_id})      # Delete user's posts first

            result = self.users_collection.delete_one({"_id": user_object_id})  # Delete the user
            return result.deleted_count > 0
        
        except Exception as e:
            print(f"Error deleting user: {e}")
            return False
        
    def close_connection(self): # Close the MongoDB connection
        self.client.close()

def display_menu(): # Displays the menu in terminal
    print("\n" + "="*40)
    print("    DATABASE MANAGER")
    print("="*40)
    print("1 CREATE USER")
    print("2 VIEW ALL USER")
    print("3 CREATE RECORD")
    print("4 VIEW USER RECORD")
    print("5 DELETE USER")
    print("6 EXIT")
    print("="*40)

def main(): #Main interactive CLI function
    try:
        db = DatabaseManager()
        print("✓ Connected to MongoDB successfully!")
    except Exception as e:
        print(f"✗ Failed to connect to MongoDB: {e}")
        print("Make sure MongoDB is running on localhost: 27017")
        return
    
    while True:
        display_menu()
        choice = input("Enter your choice (1-6): ").strip()

        if choice == '1':
            print("\n --- Create New User --- ")
            name = input("Enter name: "). strip()
            email = input("Enter email: ").strip()
            try:
                age = int(input("Enter age: ").strip())
                user_id = db.create_user(name, email, age)
                if user_id:
                    print(f"/ User created successfully! ID: {user_id}")
                else:
                    print("Failed to create user")
            except ValueError:
                print("Invalid age. Please enter a number.")

        elif choice == '2':
            print("\n --- All Users --- ")
            users = db.get_all_users()
            if users:
                for user in users:
                    print(f"ID: {user['_id']} | Name: {user['name' ]} | Email: {user['email']} | Age: {user['age' ]}")

            else:
                print("No users found.")

        elif choice == '3':
            print("\n --- Create New Post --- ")
            user_id = input("Enter user ID: ").strip()
            title = input("Enter post title: ").strip()
            content = input("Enter post content: ").strip()
            post_id = db.create_post(user_id, title, content)
            if post_id:
                print(f"✓ Post created successfully! ID: {post_id}")
            else:
                print("✗ Failed to create post")

        elif choice == '4':
            print("\n --- View User Posts --- ")
            user_id = input("Enter user ID: ").strip()
            posts = db.get_user_posts(user_id)
            if posts:
                for post in posts:
                    print(f"\nPost ID: {post['_id']}")
                    print(f"Title: {post['title']}")
                    print(f"Content: {post['content']}")
                    print(f"Created: {post['created_at']}")
                    print("-" * 30)
            else:
                print("No posts found for this user.")

        elif choice == '5':
            print("\n --- Delete User --- ")
            user_id = input("Enter user ID to delete: ").strip()
            confirm = input(f"Are you sure you want to delete user {user_id}? (y/N): ").strip().lower()
            if confirm == 'y':
                if db.delete_user(user_id):
                    print("✓ User deleted successfully!")
                else:
                    print("✗ User not found or deletion failed.")

            else:
                print("Deletion cancelled.")

        elif choice == '6':
            print("\nClosing database connection ... ")
            db.close_connection()
            print("Goodbye!")
            break

        else:
            print("✗ Invalid choice. Please enter 1-6.")

        input("\nPress Enter to continue ... ")

if __name__ == "__main__":
    main()

In [ ]:
from fastapi import FastAPI, HTTPException, status
from pydantic import BaseModel, EmailStr
from typing import List, Optional
from datetime import datetime
import sqlite3
from sql_database import DatabaseManager

app = FastAPI(title = "SQLite Database API", version = "1.0.0")

# Pydantic models for request/response
class UserCreate(BaseModel):
    name: str
    email: EmailStr
    age: int

class UserResponse(BaseModel) :
    id: int
    name: str
    email: str
    age: int
    created_at: str

class PostCreate(BaseModel):
    user_id: int
    title: str
    content: str

class PostResponse(BaseModel):
    id: int
    user_id: int
    title: str
    content: str
    created_at: str

class PostResponseForUser(BaseModel) :
    id: int
    title: str
    content: str
    created_at: str

# Initialize database
db = DatabaseManager ( )

@app.get("/")
async def root():
    return {"message": "SQLite Database API", "version": "1.0.0"}

@app.post("/users/", response_model = dict, status_code = status.HTTP_201_CREATED)
async def create_user(user: UserCreate): #Create a new user"""
    try:
        user_id = db.create_user(user.name, user.email, user.age)
        if user_id:
            return {"message": "User created successfully", "user_id": user_id}
        else:
            raise HTTPException(
            status_code = status.HTTP_400_BAD_REQUEST,
            detail = "Failed to create user. Email might already exist."
            )

    except Exception as e:
        raise HTTPException(
        status_code = status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail = f"Internal server error: {str(e)}"
        )
    
@app.get("/users/", response_model = List[UserResponse])
async def get_all_users(): # Gets everyone
    try:
        users = db.get_all_users()
        return [
            UserResponse(
                id = user_i[0],
                name = user_i[1],
                email = user_i[2],
                age = user_i[3],
                created_at = user_i[4],
            )
            for user_i in users
        ]
    except Exception as err:
        raise HTTPException(
            status_code = status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail = f"Internal server err: {str(err)}"
        )
    
@app.get("/users/{user_id}", response_model = UserResponse)
async def get_user(user_id: int): # Get use by specific ID
    try:
        with sqlite3.connect(db.db_name) as connect:
            cursor = connect.cursor()
            cursor.execute("SELECT * FROM users WHERE id = ?", (user_id,))
            user = cursor.fetchone()
        if not user:
            raise HTTPException(
                status_code = status.HTTP_404_NOT_FOUND,
                detail = "User N/A"
            )
        return UserResponse(
            id = user[0],
            name = user[1],
            email = user[2],
            age = user[3],
            created_at = user[4],
        )

    except HTTPException:
        raise
    except Exception as err:
        raise HTTPException(
            status_code = status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail = f"Internal server err: {str(err)}"
        )
    
@app.post("/posts/", response_model = dict, status_code = status.HTTP_201_CREATED)
async def create_post(post: PostCreate): # Create new post/record
    try:
        with sqlite3.connect(db.db_name) as connect:
            cursor = connect.cursor()
            cursor.execute("SELECT id FROM users WHERE id = ?", (post.user_id,))
            if not cursor.fetchone():
                raise HTTPException(
                    stauts_code = status.HTTP_404_NOT_FOUND,
                    detail  = "User N/A"
                )
        post_id = db.create_post(post.user_id, post.title, post.content)
        if post_id:
            return {"message": "Post created successfully", "post_id": post_id}
        else:
            raise HTTPException(
                status_code = status.HTTP_400_BAD_REQUEST,
                detail = "Failed to create post/record"
                )
            
    except HTTPException:
        raise
    except Exception as err:
        raise HTTPException(
            status_code = status.HTTP_500_INTERNAL_SERVER_ERROR,
            detial = f"Internal server err: {str(err)}"
        )

@app.get("/users/{user_id}/posts", response_model = List[PostResponseForUser])
async def get_user_posts(user_id: int): # Gets post/record by specific ID
    try:
        with sqlite3.connect(db.db_name) as connect:
            cursor = connect.cursor()
            cursor.execute("SELECT id FROM users WHERE id = ?", (user_id,))
            if not cursor.fetchone():
                raise HTTPException(
                    status_code = status.HTTP_404_NOT_FOUND,
                    detail = "User N/A"
                )
        posts = db.get_user_posts(user_id)
        return [
            PostResponseForUser(
                id = post_i[0],
                name = post_i[1],
                email = post_i[2],
                age = post_i[3],
                created_at = post_i[4],
            )
            for post_i in posts
        ]

    except HTTPException:
        raise
    except Exception as err:
        raise HTTPException(
            status_code = status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail = f"Internal server err: {str(err)}"
        )
    
@app.get("/posts/", response_model = List[PostResponse])
async def get_user(user_id: int): # Gets all post/record
    try:
        with sqlite3.connect(db.db_name) as connect:
            cursor = connect.cursor()
            cursor.execute("SELECT * FROM posts ORDER BY created_at DESC")
            posts = cursor.fetchone()
        return [
            UserResponse(
                id = post_i[0],
                name = post_i[1],
                email = post_i[2],
                age = post_i[3],
                created_at = post_i[4],
            )
            for post_i in posts
        ]

    except Exception as err:
        raise HTTPException(
            status_code = status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail = f"Internal server err: {str(err)}"
        )
    
@app.delete("/users/{user_id}", response_model = dict)
async def delete_user(user_id: int): # Deletes a user and their post/records
    try:
        with sqlite3.connect(db.db_name) as connect:
            cursor = connect.cursor()
            cursor.execute("SELECT id FROM users WHERE id = ?", (user_id,))
            if not cursor.fetchone():
                raise HTTPException(
                    status_code = status.HTTP_404_NOT_FOUND,
                    detail = "User N/A"
                )
        success = db.delete_user(user_id)
        if success:
            return {"message": "User deleted successfully"}
        else:
            raise HTTPException(
                status_code = status.HTTP_400_BAD_REQUEST,
                detail = "Failed to create post/record"
                )

    except HTTPException:
        raise
    except Exception as err:
        raise HTTPException(
            status_code = status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail = f"Internal server err: {str(err)}"
        )
    
@app.delete("/posts/{post_id}", response_model = dict)
async def delete_user(post_id: int): # Deletes a specific post/records
    try:
        with sqlite3.connect(db.db_name) as connect:
            cursor = connect.cursor()
            cursor.execute("DELETE FROM posts WHERE id = ?", (post_id,))
            if cursor.rowcount == 0:
                raise HTTPException(
                    status_code = status.HTTP_404_NOT_FOUND,
                    detail = "Post/Record N/A"
                )
        return {"message": "Post/Record deleted successfully"}
    
    except HTTPException:
        raise
    except Exception as err:
        raise HTTPException(
            status_code = status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail = f"Internal server err: {str(err)}"
        )
    

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host = "0.0.0.0", port = 8000)

In [ ]:
from fastapi import FastAPI, HTTPException, status
from contextlib import asynccontextmanager
from pydantic import BaseModel, EmailStr
from typing import List, Optional
from datetime import datetime
from bson.objectid import ObjectId
from sql_database import DatabaseManager
import os
from dotenv import load_dotenv

load_dotenv()

app = FastAPI(title="MongoDB Database API", version="1.0.0")

# Pydantic models for request/response
class UserCreate(BaseModel):
    name: str
    email: EmailStr
    age: int

class UserResponse(BaseModel):
    id: str
    name: str
    email: str
    age: int
    created_at: datetime

class PostCreate(BaseModel):
    user_id: str
    title: str
    content: str

class PostResponse(BaseModel):
    id: str
    user_id: str
    title: str
    content: str
    created_at: datetime

class PostResponseForUser(BaseModel):
    id: str
    title: str
    content: str
    created_at: datetime

# Initialize database
try:
    db = DatabaseManager ()
except Exception as e:
    print(f"Failed to connect to MongoDB: {e}")
    db = None

@app.on_event("startup")
async def startup_event():
    if db is None:
        raise Exception("Failed to connect to MongoDB")

@app.on_event("shutdown")
async def shutdown_event():
    if db:
        db.close_connection()

@app.get("/")
async def root():
    return {"message": "MongoDB Database API", "version": "1.0.0"}

@app.post("/users/", response_model=dict, status_code=status.HTTP_201_CREATED)
async def create_user(user: UserCreate):
    """Create a new user"""
    try:
        user_id = db.create_user(user.name, user.email, user.age)
        if user_id:
            return {"message": "User created successfully", "user_id": user_id}
        else:
            raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Failed to create user. Email might already exist."
        )

    except Exception as e:
        raise HTTPException(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail=f"Internal server error: {str(e)}"
        )
    
@app.get("/users/", response_model=List[UserResponse])
async def get_all_users():
    """Get all users"""
    try:
        users = db.get_all_users()
        return [
            UserResponse(
            id=user['_id'],
            name=user['name'],
            email=user['email'],
            age=user['age'],
            created_at=user['created_at']
            )
            for user in users
        ]
    except Exception as e:
        raise HTTPException(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail=f"Internal server error: {str(e)}"
        )
    
@app.get("/users/{user_id}", response_model=UserResponse)
async def get_user(user_id: str):
    """Get a specific user by ID"""
    try:
        if not ObjectId.is_valid(user_id):
            raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Invalid user ID format"
            )
        user = db.users_collection.find_one({"_id": ObjectId(user_id)})
        if not user:
            raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="User not found"
            )
        return UserResponse(
        id=str(user['_id']),
        name=user['name'],
        email=user['email'],
        age=user['age' ],
        created_at=user['created_at']
        )

    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail=f"Internal server error: {str(e)}"
        )
    
@app.post("/posts/", response_model=dict, status_code=status.HTTP_201_CREATED)
async def create_post(post: PostCreate):
    """Create a new post"""
    try:
        if not ObjectId.is_valid(post.user_id):
            raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Invalid user ID format"
            )
        # Check if user exists
        user = db.users_collection.find_one({"_id": ObjectId(post.user_id)})
        if not user:
            raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="User not found"
            )
        post_id = db.create_post(post.user_id, post.title, post.content)
        if post_id:
            return {"message": "Post created successfully", "post_id": post_id}
        else:
            raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Failed to create post"
            )

    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail=f"Internal server error: {str(e)}"
        )
    
@app.get("/users/{user_id}/posts", response_model=List[PostResponseForUser])
async def get_user_posts(user_id: str):
    """Get all posts by a specific user"""
    try:
        if not ObjectId.is_valid(user_id):
            raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Invalid user ID format"
            )
        # Check if user exists
        user = db.users_collection.find_one({"_id": ObjectId(user_id)})
        if not user:
            raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="User not found"
            )
        posts = db.get_user_posts(user_id)
        return [
            PostResponseForUser(
            id=post['_id'],
            title=post['title'],
            content=post['content'],
            created_at=post['created_at']
            )
            for post in posts
            ]
    
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail=f"Internal server error: {str(e)}"
        )
    
@app.get("/posts/", response_model=List[PostResponse])
async def get_all_posts():
    """Get all posts"""
    try:
        posts = list(db.posts_collection.find().sort("created_at", -1))

        # Convert ObjectId to string for response
        for post in posts:
            post['_id'] = str(post['_id'])
            post['user_id'] = str(post['user_id'])
        return [
            PostResponse(
            id=post['_id'],
            user_id=post['user_id'],
            title=post['title'],
            content=post['content'],
            created_at=post['created_at']
            )
            for post in posts
        ]
    
    except Exception as e:
        raise HTTPException(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail=f"Internal server error: {str(e)}"
        )
    

@app.delete("/users/{user_id}", response_model=dict)
async def delete_user(user_id: str):
    """Delete a user and all their posts"""
    try:
        if not ObjectId.is_valid(user_id):
            raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Invalid user ID format"
            )
        # Check if user exists
        user = db.users_collection.find_one({"_id": ObjectId(user_id)})
        if not user:
            raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="User not found"
            )

        success = db.delete_user(user_id)
        if success:
            return {"message": "User deleted successfully"}
        else:
            raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Failed to delete user"
            )

    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail=f"Internal server error: {str(e)}"
        )
    

@app.delete("/posts/{post_id}", response_model=dict)
async def delete_post(post_id: str):
    """Delete a specific post"""
    try:
        if not ObjectId.is_valid(post_id):
            raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Invalid post ID format"
            )
        result = db.posts_collection.delete_one({"_id": ObjectId(post_id)})

        if result.deleted_count == 0:
            raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Post not found"
            )
        return {"message": "Post deleted successfully"}

    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail=f"Internal server error: {str(e)}"
        )
    
@app.put("/users/{user_id}", response_model=dict)
async def update_user(user_id: str, user_update: UserCreate):
    """Update a user's information"""
    try:
        if not ObjectId.is_valid(user_id):
            raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Invalid user ID format"
            )
        # Check if user exists
        existing_user = db.users_collection.find_one({"_id": ObjectId(user_id)})
        if not existing_user:
            raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="User not found"
            )
        # Update user
        result = db.users_collection.update_one(
        {"_id": ObjectId(user_id)},
        {"$set": {
        "name": user_update.name,
        "email": user_update.email,
        "age": user_update.age
        }}
        )

        if result.modified_count > 0:
            return {"message": "User updated successfully"}
        else:
            return {"message": "No changes made to user"}

    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail=f"Internal server error: {str(e)}"
        )
    
@app.put("/posts/{post_id}", response_model=dict)
async def update_post(post_id: str, title: str, content: str):
    """Update a post's title and content"""
    try:
        if not ObjectId.is_valid(post_id):
            raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Invalid post ID format"
            )
        # Check if post exists
        existing_post = db.posts_collection.find_one({"_id": ObjectId(post_id)})
        if not existing_post:
            raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Post not found"
            )
        # Update post
        result = db.posts_collection.update_one(
                    {"_id": ObjectId(post_id)},
                    {"$set": {
                    "title": title,
                    "content": content
                    }}
        )
        if result.modified_count > 0:
            return {"message": "Post updated successfully"}
        else:
            return {"message": "No changes made to post"}
        
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(
        status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
        detail=f"Internal server error: {str(e)}"
        )
    
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host = "0.0.0.0", port = 8001)

In [1]:
from todo_SQL import db_manager, main
main()
pass


       DATABASE MANAGER
1 CREATE USER
2 CREATE NEW TASK
3 UPDATE TASK
4 VIEW TASK(S)
5 DELETE TASK
6 DELETE USER
C to Toggle 'Clear the terminal'
X to EXIT

 --- Get All Users --- 
Obj_ID: 1 | E-mail: 1 | Name: 1 | Age:  | Time created: 2025-12-14 15:25:43
Obj_ID: 3 | E-mail: 3 | Name:  | Age:  | Time created: 2025-12-14 15:51:39
Obj_ID: 4 | E-mail: 2 | Name:  | Age:  | Time created: 2025-12-14 16:05:34

 --- Get All Users --- 
Obj_ID: 1 | E-mail: 1 | Name: 1 | Age:  | Time created: 2025-12-14 15:25:43
Obj_ID: 3 | E-mail: 3 | Name:  | Age:  | Time created: 2025-12-14 15:51:39
Obj_ID: 4 | E-mail: 2 | Name:  | Age:  | Time created: 2025-12-14 16:05:34

 --- Get All Users --- 
Obj_ID: 1 | E-mail: 1 | Name: 1 | Age:  | Time created: 2025-12-14 15:25:43
Obj_ID: 3 | E-mail: 3 | Name:  | Age:  | Time created: 2025-12-14 15:51:39
Obj_ID: 4 | E-mail: 2 | Name:  | Age:  | Time created: 2025-12-14 16:05:34
✗ Invalid choice. Please enter the given options.
✗ Invalid choice. Please enter the given